### <span style="color:lightgray">EDC, November 2024</span>

# Intro to programming with LLMs
---

### Matt Hall, Equinor &nbsp; `mtha@equinor.com`

<span style="color:lightgray">&copy;2024  Matt Hall, Equinor &nbsp; | &nbsp; licensed CC BY, please share this work</span>

## Set up an environment

You will need:

- `jupyter` (if you want to run this notebook)
- `ipykernel` (if you are using Jupyter)
- `tiktoken`
- `python-dotenv` (NB, not just 'dotenv')
- `openai`

## Set up secrets

Make a file called `.env` or `secrets.txt` and give it the following contents (sort of, I will give you the correct key in the class):

```text
AZURE_OPENAI_ENDPOINT=<get it from Slack>
AZURE_OPENAI_KEY=<get it from Slack>
```

We can read environment variables from this file:

In [ ]:
# 💥 Either use a file called `.env` to store these, or
# 💥 before proceeding, add secrets.txt to .gitignore

from dotenv import load_dotenv

__ = load_dotenv("secrets.txt") # If key is in a file.

Now you can read the constants from the environment:

In [ ]:
import os

os.getenv("AZURE_OPENAI_ENDPOINT")

We can make this into a `requests` call.

## Define the client and make a request

It's usually easier to use the Python API though.

In [ ]:
from openai import AzureOpenAI


MODEL = "gpt-35-turbo" # "gpt-4o" is multimodal but more expensive.

CLIENT = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-01",
)

Here's the prompt we're going to use:

In [ ]:
prompt = "Define AI in one sentence."

<div style="border: 2px solid green; border-radius: 10px; padding: 8px; background: #DDFFDD">
<h3>EXERCISE</h3>

[Check out the docs](https://platform.openai.com/docs/overview) to figure out what a `message` object looks like and define it below. Remember to use our `prompt` string when you define it.

<a title="Look at the 'Developer quick start' > Python"><strong>Hover for a hint</strong></title>
</div>

In [ ]:
message =  # YOUR CODE HERE

response = CLIENT.chat.completions.create(
    model=MODEL,
    messages=[message],
)
response

<div style="border: 2px solid green; border-radius: 10px; padding: 8px; background: #DDFFDD">
<h3>EXERCISE</h3>

Extract the plain text answer to your question.

<a title="You are looking for the attribute called choices > message > content"><strong>Hover for a hint</strong></title>
</div>

In [ ]:
# YOUR CODE HERE



<div style="border: 2px solid green; border-radius: 10px; padding: 8px; background: #DDFFDD">
<h3>EXERCISE</h3>

Write a function called `ask()` to contain this code.
</div>

In [ ]:
def ask():
    # YOUR CODE HERE

ask("Try it out!")

## A tokenizer

We can use `tiktoken` for tokenization.

In [ ]:
import tiktoken

def tokenize(prompt):
    encoding = tiktoken.encoding_for_model(MODEL)
    tokens = encoding.encode(prompt)
    decode = lambda token: encoding.decode_single_token_bytes(token).decode()
    return [decode(token) for token in tokens]

tokenize("Stratigraphically.")

## Embeddings

Embedding models are learned during training of the LLM. 

In [ ]:
def get_embedding(text, model="text-embedding-3-large"):
    text = text.replace("\n", " ")
    response = CLIENT.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

e = get_embedding("Equinor is an energy company.")
len(e)

In [ ]:
e[:10]

## Conversations

We can fake a conversation by storing the chat 'steps' and passing them back to the model on each new request.

In [ ]:
class Convo:
    def __init__(self, temperature=0, model='gpt-35-turbo'):
        self.temperature = temperature
        self.model = model
        self.messages = []

    def ask(self, prompt):
        self.messages.append({"role": "user", "content": prompt})
        response = CLIENT.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            max_tokens=1024,
            messages=self.messages
        )
        content = response.choices[0].message.content
        self.messages.append({'role': 'assistant',  'content': content})
        return content

    def history(self):
        return self.messages

In [ ]:
convo = Convo()
convo.ask("I'm Matt, who are you?")

In [ ]:
convo.ask("What's my name?")

In [ ]:
convo.history()

## Use the REST API

We can use the web API directly, for example with Python's `requests` library. **NOTE** You might need to 

Here is the `curl` example from [the docs](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line%2Cjavascript-keyless%2Ctypescript-keyless%2Cpython-new&pivots=rest-api#rest-api):

```sh
curl $AZURE_OPENAI_ENDPOINT/openai/deployments/gpt-35-turbo/chat/completions?api-version=2024-02-01 \
  -H "Content-Type: application/json" \
  -H "api-key: $AZURE_OPENAI_API_KEY" \
  -d '{"messages":[{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Define AI in one sentence."}]}'
```

In [ ]:
import requests

prompt = "Define AI in one sentence."

headers = {
    "Content-Type": "application/json",
    "api-key": os.getenv("AZURE_OPENAI_KEY"),
}

model = "gpt-35-turbo"
base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
url = f"{base_url}/openai/deployments/{model}/chat/completions"

params = {"api-version": "2024-02-01"}

json = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {'role': 'user', 'content': "I'm Matt, who are you?"},
        {'role': 'assistant', 'content': 'Hello Matt, I am an AI digital assistant. How can I assist you today?'},
        {'role': 'user', 'content': "What's my name?"},
    ]
}

r = requests.post(url, params=params, headers=headers, json=json)
r.status_code

In [ ]:
r.json()

## Adding tools to the chat

See [Using tools](./Using_tools.ipynb)

## Including images in the context

See [Sending images](./Sending_images.ipynb)

<span style="color:lightgray">&copy; 2024 Matt Hall, Equinor &nbsp; | &nbsp; licensed CC BY, please share this work</span>